# Описание проекта

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# 1. Загрузка и подготовка данных

In [1]:
# Отключим предупреждения
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState

In [3]:
MY_RANDOM_STATE = 12345

In [4]:
#data1 = pd.read_csv('geo_data_0.csv')
#data2 = pd.read_csv('geo_data_1.csv')
#data3 = pd.read_csv('geo_data_2.csv')
data1 = pd.read_csv('/datasets/geo_data_0.csv')
data2 = pd.read_csv('/datasets/geo_data_1.csv')
data3 = pd.read_csv('/datasets/geo_data_2.csv')
print(data1.info())
print(data2.info())
print(data3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory 

In [5]:
data1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [6]:
# В дальнейшем для работы модели признак id нам не важен. Удалим его из наборов данных.
data1 = data1.drop('id', axis=1)
data2 = data2.drop('id', axis=1)
data3 = data3.drop('id', axis=1)

# 2. Обучение и проверка модели

### Для каждого региона (данных dataN) необходимо:
* разбить данные на обучающую и валидационную выборки в соотношении 75:25
* обучить модель и сделать предсказания на валидационной выборке


In [7]:
# Подготовимся к масштабированию
numeric = ['f0', 'f1', 'f2']
scaler = StandardScaler()

# Процедура делит данные на трейн и вэлид, обучает модель и делает предсказания
# Возвращает правильные ответы и предсказания
def process_data(data):
#     Делим данные
    x_data = data.drop('product', axis=1)
    y_data = data['product']
    x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.25, random_state=MY_RANDOM_STATE)
    
#     Масштабируем числовые признаки
    scaler.fit(x_train[numeric])
    x_train[numeric] = scaler.transform(x_train[numeric])
    x_valid[numeric] = scaler.transform(x_valid[numeric])

#     Модель делает предсказания
    model = LinearRegression()
    model.fit(x_train, y_train)
    y_pred = model.predict(x_valid)
    
# Для дальнейшей работы необходимо, чтобы обе переменные (y_valid, y_pred) 
# были типа Series причем с одинаковыми индексами (от 0 до 24999)
    y_valid = pd.Series(y_valid.to_list())
    y_pred = pd.Series(y_pred)

    return y_valid, y_pred

### Для каждого месторождения применим написанную функцию, отдельно сохраним предсказания и правильные ответы, напечатаем средний запас сырья и RMSE модели

In [8]:
print('Регион 1:')
y_valid1, y_pred1 = process_data(data1)
print('mean:', y_valid1.mean())
print('rmse:', mean_squared_error(y_valid1, y_pred1) ** 0.5)

print('Регион 2:')
y_valid2, y_pred2 = process_data(data2)
print('mean:', y_valid2.mean())
print('rmse:', mean_squared_error(y_valid2, y_pred2) ** 0.5)

print('Регион 3:')
y_valid3, y_pred3 = process_data(data3)
print('mean:', y_valid3.mean())
print('rmse:', mean_squared_error(y_valid3, y_pred3) ** 0.5)

Регион 1:
mean: 92.07859674082927
rmse: 37.5794217150813
Регион 2:
mean: 68.72313602435997
rmse: 0.8930992867756158
Регион 3:
mean: 94.88423280885438
rmse: 40.02970873393434


### Вывод

1. В среднем в регионе 3 месторождения самые богатые нефтью (94,88 тыс. бар.)
2. В регионе 2 метрика RMSE близка к нулю (всего 0,89). То есть модель очень хорошо предсказывает залежи нефти по 2 региону.

# 3. Подготовка к расчёту прибыли

### 3.1. Сохраните в коде все ключевые значения для расчётов.

In [9]:
# бюджет на разработку месторождений — 10 млрд рублей
DEV_BUDGET = 10 * 10**9

# стоимость бурения одной скважины — 50 млн рублей
DRILLING_COST = 50 * 10**6

# один баррель сырья приносит 4500 рублей прибыли
PROFIT_PER_BARREL = 4500

# При разведке региона проводится исследование 500 точек
POINTS_TO_EXPLORE = 500

# Дополнительно посчитаем на бурение скольких точек в принципе хватает выделенного бюджета
POINTS_TO_DRILL = DEV_BUDGET // DRILLING_COST
print(POINTS_TO_DRILL)

200


### 3.2. Посчитайте минимальный средний объём сырья в месторождениях региона, достаточный для его разработки. Напишите выводы.

In [10]:
print('Минимальный объем сырья достаточный для разработки:')
print(DRILLING_COST / PROFIT_PER_BARREL)

Минимальный объем сырья достаточный для разработки:
11111.111111111111


Во всех регионах средних объем сырья в месторождениях больше чем минимально необходимый.

### 3.3. Напишите функцию для расчёта прибыли по набору отобранных месторождений и предсказаний модели.

In [11]:
# Для удобства прибыль будем считать в миллионах рублей
def get_revenue(y_sample):
    result = (y_sample.sum() * PROFIT_PER_BARREL * 1000 - DEV_BUDGET) / 10**6
    return result

# 4. Расчёт прибыли и рисков 

### 4.1. Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.

Для каждого региона мы 1000 раз сделаем следующее:
* получим случайную выборку из 500 точек (по условию "при разведке региона проводится исследование 500 точек")
* по полученной ранее модели предскажем объем запасов в каждом из месторождений
* выберем 200 местрождений с максимальным объемом запасов
* рассчитаем суммарную прибыль исходя из выборки с помощью написанной ранее функции

In [12]:
state = RandomState(MY_RANDOM_STATE)

def get_profit_sample(y_valid, y_pred):
    result = []
    for i in range(1000):
        y_pred_sample = y_pred.sample(n=POINTS_TO_EXPLORE, replace=True, random_state=state).sort_values(ascending=False)
        y_valid_sample = y_valid[y_pred_sample.index][:POINTS_TO_DRILL]
        revenue = get_revenue(y_valid_sample)
        result.append(revenue)
    return pd.Series(result)

### 4.2. Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков.

In [13]:
# Напишем функцию для получения всех необходимых показателей:
# средняя прибыль
# границы 95% доверительного интервала
# риск убытков (т.е. долю месторождений с отрицательной прибылью)
def squeeze_indicators(profit_sample):
    mean_profit = profit_sample.mean()
    lower_endpoint = profit_sample.quantile(q=0.025)
    upper_endpoint = profit_sample.quantile(q=0.975)
    loss_prob = profit_sample[profit_sample < 0].count() / len(profit_sample)
    return mean_profit, lower_endpoint, upper_endpoint, loss_prob

In [14]:
profit_sample1 = get_profit_sample(y_valid1, y_pred1)
mean_profit1, lower_endpoint1, upper_endpoint1, loss_prob1 = squeeze_indicators(profit_sample1)

profit_sample2 = get_profit_sample(y_valid2, y_pred2)
mean_profit2, lower_endpoint2, upper_endpoint2, loss_prob2 = squeeze_indicators(profit_sample2)

profit_sample3 = get_profit_sample(y_valid3, y_pred3)
mean_profit3, lower_endpoint3, upper_endpoint3, loss_prob3 = squeeze_indicators(profit_sample3)

In [15]:
print('|{:<15}|{:<15}|{:<15}|{:<15}|{:<15}|'.format('Регион', 'Сред.прибыль', 'Ниж.гран.ДИ', 'Верх.гран.ДИ', 'Риск убытков'))
print('|{:<15}|{:>15.2f}|{:>15.2f}|{:>15.2f}|{:>15}|'.format('1', mean_profit1, lower_endpoint1, upper_endpoint1, loss_prob1))
print('|{:<15}|{:>15.2f}|{:>15.2f}|{:>15.2f}|{:>15}|'.format('2', mean_profit2, lower_endpoint2, upper_endpoint2, loss_prob2))
print('|{:<15}|{:>15.2f}|{:>15.2f}|{:>15.2f}|{:>15}|'.format('3', mean_profit3, lower_endpoint3, upper_endpoint3, loss_prob3))

|Регион         |Сред.прибыль   |Ниж.гран.ДИ    |Верх.гран.ДИ   |Риск убытков   |
|1              |       93961.65|       88887.84|       99097.67|            0.0|
|2              |       94611.56|       90780.51|       98629.52|            0.0|
|3              |       93929.50|       88877.72|       99345.63|            0.0|


### Вывод: ориентируясь на показатели средней прибыль рекомендую руководству компании начать работу в регионе 2